# Python Implementation of RSA, Digital Signature and Certificates
> Implementation of RSA, Digital Signature and Certificates without divining too deep into the Mathematics; then list out key math principles for further reference. In this way, one can understand *what* and *how* as a context before drilling into *why*.

- toc: true
- branch: master
- badges: false
- comments: true
- categories: [jupyter, python, math]
- hide: false
- search_exclude: true
- metadata_key1: metadata_value1
- metadata_key2: metadata_value2

## Introduction

Information is power. However, to transmit info is tricky. At least 2 issues need to be solved in information transmission: **Security** and **Integrity**.

- **Security (RSA)**

No one can see the naked message other than the one intended.

- **Integrity (Digital Signature)**

No one shall replace the message with another dummy one in the middle of transimission;  
If it happened, however, intended message receiver shall be able to find out.

## Rivest, Shamir and Adleman (RSA) Cryptosystem

![rsa](rsa_imgs/bg2013062702.jpg)
@MIT

### Helper Functions

In [57]:
#collapse-hide
import random
import base64
from hashlib import sha256

#### Check If a Number is Prime

In [2]:
#collapse-show
def check_prime(num):
    """Check if num is prime
    
    """
    # prime numbers are greater than 1
    if num > 1:
        # check for factors
        # TODO: refactoring to parallel
        for i in range(2, num):
            if (num % i) == 0:
                return False
        else:
            return True
    # if input number is less than
    # or equal to 1, it is not prime
    else:
        return False

#### Modular Multiplicative Inverse Function

In [3]:
#collapse-show
# https://stackoverflow.com/a/9758173/3317548
def egcd(a, b):
    if a == 0:
        return (b, 0, 1)
    else:
        g, y, x = egcd(b % a, a)
        return (g, x - (b // a) * y, y)


def modinv(a, m):
    g, x, y = egcd(a, m)
    if g != 1:
        raise Exception("modular inverse does not exist")
    else:
        aa = x % m
        bb = int((1 - a * aa) / m)
        return aa, bb

#### List All Prime Numbers Smaller Than a Value

In [4]:
#collapse-show
# https://stackoverflow.com/q/2068372/3317548
def find_prime(n):
    """ Returns  a list of primes < n """
    sieve = [True] * (n // 2)
    for i in range(3, int(n ** 0.5) + 1, 2):
        if sieve[i // 2]:
            sieve[i * i // 2 :: i] = [False] * ((n - i * i - 1) // (2 * i) + 1)
    return [2] + [2 * i + 1 for i in range(1, n // 2) if sieve[i]]

###  Encryption & Decryption Process

- Receiver (Richael) generate a pair of keys: **public** & **private**
- She keeps **private** key to herself confidentially
- She sends multiple copies of **public** key to every of her friends who need to send her message
- Any one got Richael's **public** key can send her message now. For instance, Stephen:  
    - Stephen'd like to send the original message: "Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 :)", to Richael.
    - Stephen'd encrypt the message with Richaels **public** key into something looks random, e.g.  "MHgxNTMzCjB4MmFmNgoweGFiOA...YWE0CjB4MWZlMwoweDIzMGY="
    - Stephen'd send that "random" stuff to Richael.
- After receiving the "random" stuff from Stephen, Richael uses her own **private** key to decrypt it into orignal message "Hello Richael. How are you?!"
- Only with Richael's **private** key, the "random" stuff, produced with Richael's **public** key, can be decrpted. Unless Richael shares her **private** key, which she shall never do, no one other than Richael herself can obtain the original message "Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 :)"
- If Richael'd like to answer to Stephen, she must obtain Stephen's **public** key.

#### Key Pair: Public & Private (Richael's Task)

- Randomly pick 2 prime numbers (larger the better): **p** & **q** 
- Got their muliplication: **n=pxq**
- Got the Euler number: **phi=(p-1)x(q-1)**
- Randomly find a prime number in the range of 1 to **phi** (larger the better): **e**
- Find the Modular Multiplicative Inverse of **e** relative to **phi**: **k**
- public key is: **(n, e)**
- private key is: **(n, k)**

In [5]:
# prime numbers
p = random.choice(list(find_prime(1024)))
q = random.choice(list(find_prime(1024)))
assert check_prime(p)
assert check_prime(q)
n = p * q
p, q, n, len(bin(n)), bin(n)

(853, 499, 425647, 21, '0b1100111111010101111')

In [6]:
# Euler number
phi = (p - 1) * (q - 1)
phi

424296

In [7]:
e = random.choice(find_prime(phi))
assert check_prime(e)
e

314953

In [8]:
k = modinv(e, phi)[0]
k

112369

In [9]:
public_key, private_key = (n, e), (n, k)
public_key, private_key

((425647, 314953), (425647, 112369))

#### Encryption & Decryption Computation

- Original message to unicode numbers: **msg**
- Encrypt message: **msg_encrypt = (msg^e)%n**
- Decrypt message: **msg = (msg_encrypt^k)%n**

#### Origial Message (Stephen's Task)

In [61]:
#collapse-output
# unicode
msg = [ord(i) for i in "Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)"]
msg

[72,
 101,
 108,
 108,
 111,
 32,
 82,
 105,
 99,
 104,
 97,
 101,
 108,
 46,
 32,
 76,
 101,
 116,
 39,
 115,
 32,
 109,
 101,
 101,
 116,
 32,
 97,
 116,
 32,
 54,
 58,
 50,
 53,
 32,
 112,
 109,
 32,
 116,
 111,
 109,
 111,
 114,
 114,
 111,
 119,
 32,
 97,
 116,
 32,
 116,
 104,
 101,
 32,
 97,
 105,
 114,
 112,
 111,
 114,
 116,
 32,
 71,
 97,
 116,
 101,
 53,
 44,
 32,
 84,
 101,
 114,
 109,
 105,
 110,
 97,
 108,
 32,
 50,
 32,
 45,
 45,
 83,
 116,
 101,
 112,
 104,
 101,
 110,
 32,
 58,
 41]

#### Encrption (Stephen's Task)

In [62]:
#collapse-output
# encrption
[(val ** public_key[1]) % public_key[0] for val in msg]

[197325,
 411465,
 418365,
 418365,
 246125,
 177580,
 222918,
 219180,
 84094,
 263749,
 89156,
 411465,
 418365,
 229863,
 177580,
 308909,
 411465,
 44601,
 294261,
 352422,
 177580,
 24787,
 411465,
 411465,
 44601,
 177580,
 89156,
 44601,
 177580,
 396892,
 318333,
 155084,
 308030,
 177580,
 374496,
 24787,
 177580,
 44601,
 246125,
 24787,
 246125,
 324521,
 324521,
 246125,
 422335,
 177580,
 89156,
 44601,
 177580,
 44601,
 263749,
 411465,
 177580,
 89156,
 219180,
 324521,
 374496,
 246125,
 324521,
 44601,
 177580,
 104344,
 89156,
 44601,
 411465,
 308030,
 289371,
 177580,
 397962,
 411465,
 324521,
 24787,
 219180,
 49482,
 89156,
 418365,
 177580,
 155084,
 177580,
 3523,
 3523,
 372567,
 44601,
 411465,
 374496,
 263749,
 411465,
 49482,
 177580,
 318333,
 39481]

In [63]:
# hex encode or something else
encry_msg = '\n'.join([hex((val ** public_key[1]) % public_key[0]) for val in msg])
encry_msg

'0x302cd\n0x64749\n0x6623d\n0x6623d\n0x3c16d\n0x2b5ac\n0x366c6\n0x3582c\n0x1487e\n0x40645\n0x15c44\n0x64749\n0x6623d\n0x381e7\n0x2b5ac\n0x4b6ad\n0x64749\n0xae39\n0x47d75\n0x560a6\n0x2b5ac\n0x60d3\n0x64749\n0x64749\n0xae39\n0x2b5ac\n0x15c44\n0xae39\n0x2b5ac\n0x60e5c\n0x4db7d\n0x25dcc\n0x4b33e\n0x2b5ac\n0x5b6e0\n0x60d3\n0x2b5ac\n0xae39\n0x3c16d\n0x60d3\n0x3c16d\n0x4f3a9\n0x4f3a9\n0x3c16d\n0x671bf\n0x2b5ac\n0x15c44\n0xae39\n0x2b5ac\n0xae39\n0x40645\n0x64749\n0x2b5ac\n0x15c44\n0x3582c\n0x4f3a9\n0x5b6e0\n0x3c16d\n0x4f3a9\n0xae39\n0x2b5ac\n0x19798\n0x15c44\n0xae39\n0x64749\n0x4b33e\n0x46a5b\n0x2b5ac\n0x6128a\n0x64749\n0x4f3a9\n0x60d3\n0x3582c\n0xc14a\n0x15c44\n0x6623d\n0x2b5ac\n0x25dcc\n0x2b5ac\n0xdc3\n0xdc3\n0x5af57\n0xae39\n0x64749\n0x5b6e0\n0x40645\n0x64749\n0xc14a\n0x2b5ac\n0x4db7d\n0x9a39'

In [64]:
# base64 encode
encry_msg = base64.b64encode(encry_msg.encode('utf8'))
encry_msg

b'MHgzMDJjZAoweDY0NzQ5CjB4NjYyM2QKMHg2NjIzZAoweDNjMTZkCjB4MmI1YWMKMHgzNjZjNgoweDM1ODJjCjB4MTQ4N2UKMHg0MDY0NQoweDE1YzQ0CjB4NjQ3NDkKMHg2NjIzZAoweDM4MWU3CjB4MmI1YWMKMHg0YjZhZAoweDY0NzQ5CjB4YWUzOQoweDQ3ZDc1CjB4NTYwYTYKMHgyYjVhYwoweDYwZDMKMHg2NDc0OQoweDY0NzQ5CjB4YWUzOQoweDJiNWFjCjB4MTVjNDQKMHhhZTM5CjB4MmI1YWMKMHg2MGU1YwoweDRkYjdkCjB4MjVkY2MKMHg0YjMzZQoweDJiNWFjCjB4NWI2ZTAKMHg2MGQzCjB4MmI1YWMKMHhhZTM5CjB4M2MxNmQKMHg2MGQzCjB4M2MxNmQKMHg0ZjNhOQoweDRmM2E5CjB4M2MxNmQKMHg2NzFiZgoweDJiNWFjCjB4MTVjNDQKMHhhZTM5CjB4MmI1YWMKMHhhZTM5CjB4NDA2NDUKMHg2NDc0OQoweDJiNWFjCjB4MTVjNDQKMHgzNTgyYwoweDRmM2E5CjB4NWI2ZTAKMHgzYzE2ZAoweDRmM2E5CjB4YWUzOQoweDJiNWFjCjB4MTk3OTgKMHgxNWM0NAoweGFlMzkKMHg2NDc0OQoweDRiMzNlCjB4NDZhNWIKMHgyYjVhYwoweDYxMjhhCjB4NjQ3NDkKMHg0ZjNhOQoweDYwZDMKMHgzNTgyYwoweGMxNGEKMHgxNWM0NAoweDY2MjNkCjB4MmI1YWMKMHgyNWRjYwoweDJiNWFjCjB4ZGMzCjB4ZGMzCjB4NWFmNTcKMHhhZTM5CjB4NjQ3NDkKMHg1YjZlMAoweDQwNjQ1CjB4NjQ3NDkKMHhjMTRhCjB4MmI1YWMKMHg0ZGI3ZAoweDlhMzk='

#### Decryption (Richael's Task)

In [65]:
#collapse-output
# base64 decode and hex decode
[int(i, 16) for i in base64.b64decode(encry_msg).decode('utf8').split('\n')]

[197325,
 411465,
 418365,
 418365,
 246125,
 177580,
 222918,
 219180,
 84094,
 263749,
 89156,
 411465,
 418365,
 229863,
 177580,
 308909,
 411465,
 44601,
 294261,
 352422,
 177580,
 24787,
 411465,
 411465,
 44601,
 177580,
 89156,
 44601,
 177580,
 396892,
 318333,
 155084,
 308030,
 177580,
 374496,
 24787,
 177580,
 44601,
 246125,
 24787,
 246125,
 324521,
 324521,
 246125,
 422335,
 177580,
 89156,
 44601,
 177580,
 44601,
 263749,
 411465,
 177580,
 89156,
 219180,
 324521,
 374496,
 246125,
 324521,
 44601,
 177580,
 104344,
 89156,
 44601,
 411465,
 308030,
 289371,
 177580,
 397962,
 411465,
 324521,
 24787,
 219180,
 49482,
 89156,
 418365,
 177580,
 155084,
 177580,
 3523,
 3523,
 372567,
 44601,
 411465,
 374496,
 263749,
 411465,
 49482,
 177580,
 318333,
 39481]

In [66]:
# RSA decode
decry_msg = [
    int(i, 16) ** private_key[1] % private_key[0]
    for i in base64.b64decode(encry_msg).decode("utf8").split("\n")
]
print(decry_msg)
print("".join([chr(i) for i in decry_msg]))

[72, 101, 108, 108, 111, 32, 82, 105, 99, 104, 97, 101, 108, 46, 32, 76, 101, 116, 39, 115, 32, 109, 101, 101, 116, 32, 97, 116, 32, 54, 58, 50, 53, 32, 112, 109, 32, 116, 111, 109, 111, 114, 114, 111, 119, 32, 97, 116, 32, 116, 104, 101, 32, 97, 105, 114, 112, 111, 114, 116, 32, 71, 97, 116, 101, 53, 44, 32, 84, 101, 114, 109, 105, 110, 97, 108, 32, 50, 32, 45, 45, 83, 116, 101, 112, 104, 101, 110, 32, 58, 41]
Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)


### Why is this secure?

![Euler](rsa_imgs/euler.png)
@Euler  
*He changed the world and he is now everywhere; Remeber Euler and everything about him, otherwise you'd fail at some stage of your life, some math exams for sure.*

- If another friend of Richael, say Tom, would like to know what Stephen sent to Richael, without asking Stephen or Richael.
- Another guy, say Joseph, was eavesdropping the network communication.
- Either Tom or Joseph would already have access to information of: 
    - **n** and **e** from public key because Richael broadcast it on the network to her friends.
    - **msg_encrypt** because Stephen put it on the network for Richael to retrieve.
- In order to decrypt **msg_encrypt**, they need **private key** **(n, k)**;
- They already have **n** so only need to figure out **k**.
- **k** was generated with **e** and **phi**.
- They know **e**. So they only need to know *phi*. 
- They need **p** and **q** to know **phi**
- They know **n** which is the multiplication of prime numbers **p** and **q**
- Tom or Joseph need to do **Prime Factorization** as **n=pxq**
- Given **n** is large enough (1024 bits in nowadays common sense), Tom and Joseph would had been long gone before a modern computer, even they have access to [Fugaku](https://en.wikipedia.org/wiki/TOP500), could manage to figure out which 2 prime numebrs Richael picked at the first place, let along about by "6:25 pm tomrrow".
- Richael may choose to update her key pairs frequently (change to different **p** and **q**) for the next message; this means Tom and Joseph can never catch up without a [Quantum Computer](https://www.newscientist.com/article/2227387-quantum-computer-sets-new-record-for-finding-prime-number-factors/) on their hands.

### Investigate and Understand Your Computer

Now if go to `~/.ssh`, you shall identify a number of different key files. Some of them are private key whilst others are public key; some are generated by OpenSSH whilst others by OpenSSL. They are also encoded by ASN.1 format among some variates.

- You can convert it into `*.PEM` file via:  

`ssh-keygen -f YOUR_KEY.pub -e -m pem > YOUR_KEY.pub.pem`

- Then inspect the content via:

```python
from Crypto.PublicKey import RSA
from base64 import b64decode

pem_key = b"MII<DO NOT SHARE PRIVATE KEY>="
key = b64decode(pem_key)
keyPriv = RSA.importKey(key)
# key now has all the components of the private
print(keyPriv.keydata)
print(keyPriv.n, keyPriv.e, keyPriv.d, keyPriv.p, keyPriv.q, keyPriv.u)

assert keyPriv.p * keyPriv.q == keyPriv.n
```

For public key `*.pub.pem`, it has only `keyPriv.n` and `keyPriv.e`.   
See: https://www.dlitz.net/software/pycrypto/api/2.6/Crypto.PublicKey.RSA-module.html

- Similarly, if you check `less ~/.ssh/known_hosts`, you shall see a number of lines of records, composing of:  

`<domain_name,ip_address> <algorithm, e.g. ssh-rsa> <public_key>`  

You can convert those `public_key` into `*.pem` like above and use the code snippet to check its content. You shall got `n` and `e`. When you send message to those servers, your message is encrypted with those `n` and `e`, and those server will use their private key to decrypt your message.

## Digital Signature

What if Joseph pretends to be Stephen, and sends Richael another message encrypted with Richaels public key, saying "Hello Richael. Let's meet at 6:25 am the day after tomorrow at the train station Gate 1 :)". How does Richael know the message is truly from Stephen, or being altered by Joseph? 

Another case would be Joseph intercepted Stephen's **msg_encrypt**, then modified/deleted/added some characters in that *random* stuff ("MHgxNTMzCjB4MmFmNgoweGFiO>B<...YWE0CjB4MWZlMwoweDIzMGY="), before sending to Rachael for decryption? Even though he cannot decrypt the message without Richael's private key, he can still alter it. For example, hiding some information about location:

In [47]:
encry_msg = base64.b64encode(
    "\n".join(
        base64.b64decode(
            b"MHgzMDJjZAoweDY0NzQ5CjB4NjYyM2QKMHg2NjIzZAoweDNjMTZkCjB4MmI1YWMKMHgzNjZjNgoweDM1ODJjCjB4MTQ4N2UKMHg0MDY0NQoweDE1YzQ0CjB4NjQ3NDkKMHg2NjIzZAoweDM4MWU3CjB4MmI1YWMKMHg0YjZhZAoweDY0NzQ5CjB4YWUzOQoweDQ3ZDc1CjB4NTYwYTYKMHgyYjVhYwoweDYwZDMKMHg2NDc0OQoweDY0NzQ5CjB4YWUzOQoweDJiNWFjCjB4MTVjNDQKMHhhZTM5CjB4MmI1YWMKMHg2MGU1YwoweDRkYjdkCjB4MjVkY2MKMHg0YjMzZQoweDJiNWFjCjB4NWI2ZTAKMHg2MGQzCjB4MmI1YWMKMHhhZTM5CjB4M2MxNmQKMHg2MGQzCjB4M2MxNmQKMHg0ZjNhOQoweDRmM2E5CjB4M2MxNmQKMHg2NzFiZgoweDJiNWFjCjB4MTVjNDQKMHhhZTM5CjB4MmI1YWMKMHhhZTM5CjB4NDA2NDUKMHg2NDc0OQoweDJiNWFjCjB4MTVjNDQKMHgzNTgyYwoweDRmM2E5CjB4NWI2ZTAKMHgzYzE2ZAoweDRmM2E5CjB4YWUzOQoweDJiNWFjCjB4MTk3OTgKMHgxNWM0NAoweGFlMzkKMHg2NDc0OQoweDRiMzNlCjB4NDZhNWIKMHgyYjVhYwoweDYxMjhhCjB4NjQ3NDkKMHg0ZjNhOQoweDYwZDMKMHgzNTgyYwoweGMxNGEKMHgxNWM0NAoweDY2MjNkCjB4MmI1YWMKMHgyNWRjYwoweDJiNWFjCjB4NGRiN2QKMHg1GFM5"
        )
        .decode("utf8")
        .split("\n")[:33]
    ).encode("utf8")
)
encry_msg

b'MHgzMDJjZAoweDY0NzQ5CjB4NjYyM2QKMHg2NjIzZAoweDNjMTZkCjB4MmI1YWMKMHgzNjZjNgoweDM1ODJjCjB4MTQ4N2UKMHg0MDY0NQoweDE1YzQ0CjB4NjQ3NDkKMHg2NjIzZAoweDM4MWU3CjB4MmI1YWMKMHg0YjZhZAoweDY0NzQ5CjB4YWUzOQoweDQ3ZDc1CjB4NTYwYTYKMHgyYjVhYwoweDYwZDMKMHg2NDc0OQoweDY0NzQ5CjB4YWUzOQoweDJiNWFjCjB4MTVjNDQKMHhhZTM5CjB4MmI1YWMKMHg2MGU1YwoweDRkYjdkCjB4MjVkY2MKMHg0YjMzZQ=='

In [48]:
# RSA decode
decry_msg = [
    int(i, 16) ** private_key[1] % private_key[0]
    for i in base64.b64decode(encry_msg).decode("utf8").split("\n")
]
print(decry_msg)
print("".join([chr(i) for i in decry_msg]))

[72, 101, 108, 108, 111, 32, 82, 105, 99, 104, 97, 101, 108, 46, 32, 76, 101, 116, 39, 115, 32, 109, 101, 101, 116, 32, 97, 116, 32, 54, 58, 50, 53]
Hello Richael. Let's meet at 6:25


### Two Way RSA

Previously, we introduced only one-way RSA: everyone has Richael's **public** key. But Richael can also have others' **public** key, especially, the message sender Stephen's: 

In [59]:
# Stephen generate his key pair
p = random.choice(list(find_prime(1024)))
q = random.choice(list(find_prime(1024)))
assert check_prime(p)
assert check_prime(q)
n = p * q
p, q, n, len(bin(n)), bin(n)

# Euler number
phi = (p - 1) * (q - 1)
phi

e = random.choice(find_prime(phi))
assert check_prime(e)
e

k = modinv(e, phi)[0]
k

public_key_stephen, private_key_stephen = (n, e), (n, k)
public_key_stephen, private_key_stephen

((107221, 89657), (107221, 94929))

### Hash

Stephen will keep his **private** key to himself, but share the **public** key to his friends, including Richael. Before Stephen sends the message to Richael, he will also generate an **abstract** of the total message, e.g. with Hash:

In [67]:
signature = sha256(b"Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)").hexdigest()
signature

'22edc929d9211d1102f7af88b85ea472c9e64629074985cba94607526dff3029'

Now Stephen will encrypt this abstract with his **private** key:

In [72]:
# hex encode or something else
encry_msg = "\n".join(
    [
        hex((ord(val) ** private_key_stephen[1]) % private_key_stephen[0])
        for val in signature
    ]
)

# base64 encode
encry_msg = base64.b64encode(encry_msg.encode("utf8"))
encry_msg

b'MHgxOTQxMAoweDE5NDEwCjB4MTRiYWIKMHg2MTZjCjB4ZGFhCjB4MTJkMmIKMHgxOTQxMAoweDEyZDJiCjB4NjE2YwoweDEyZDJiCjB4MTk0MTAKMHg2OGEzCjB4NjhhMwoweDYxNmMKMHg2OGEzCjB4NjhhMwoweDE1MTA5CjB4MTk0MTAKMHgxNGEyNAoweGU4MWYKMHgxNDIyOAoweDE0YTI0CjB4MTk2NmQKMHgxOTY2ZAoweGIyMGEKMHgxOTY2ZAoweDE2YzY5CjB4MTRiYWIKMHgxNDIyOAoweDExMjYyCjB4ZTgxZgoweDE5NDEwCjB4ZGFhCjB4MTJkMmIKMHgxNGJhYgoweDE5MTI5CjB4MTEyNjIKMHgxOTEyOQoweDE5NDEwCjB4MTJkMmIKMHgxNTEwOQoweGU4MWYKMHgxMTI2MgoweDEyZDJiCjB4MTk2NmQKMHgxNmM2OQoweGRhYQoweGIyMGEKMHgxNDIyOAoweDEyZDJiCjB4MTEyNjIKMHgxOTEyOQoweDE1MTA5CjB4ZTgxZgoweDE2YzY5CjB4MTk0MTAKMHgxOTEyOQoweDYxNmMKMHgxNGEyNAoweDE0YTI0CjB4MTk1YTkKMHgxNTEwOQoweDE5NDEwCjB4MTJkMmI='

Finally, the message ready to be shipped to Richael by Stephen becomes:

In [75]:
message_to_Richael = {
    "msg": b"MHgzMDJjZAoweDY0NzQ5CjB4NjYyM2QKMHg2NjIzZAoweDNjMTZkCjB4MmI1YWMKMHgzNjZjNgoweDM1ODJjCjB4MTQ4N2UKMHg0MDY0NQoweDE1YzQ0CjB4NjQ3NDkKMHg2NjIzZAoweDM4MWU3CjB4MmI1YWMKMHg0YjZhZAoweDY0NzQ5CjB4YWUzOQoweDQ3ZDc1CjB4NTYwYTYKMHgyYjVhYwoweDYwZDMKMHg2NDc0OQoweDY0NzQ5CjB4YWUzOQoweDJiNWFjCjB4MTVjNDQKMHhhZTM5CjB4MmI1YWMKMHg2MGU1YwoweDRkYjdkCjB4MjVkY2MKMHg0YjMzZQoweDJiNWFjCjB4NWI2ZTAKMHg2MGQzCjB4MmI1YWMKMHhhZTM5CjB4M2MxNmQKMHg2MGQzCjB4M2MxNmQKMHg0ZjNhOQoweDRmM2E5CjB4M2MxNmQKMHg2NzFiZgoweDJiNWFjCjB4MTVjNDQKMHhhZTM5CjB4MmI1YWMKMHhhZTM5CjB4NDA2NDUKMHg2NDc0OQoweDJiNWFjCjB4MTVjNDQKMHgzNTgyYwoweDRmM2E5CjB4NWI2ZTAKMHgzYzE2ZAoweDRmM2E5CjB4YWUzOQoweDJiNWFjCjB4MTk3OTgKMHgxNWM0NAoweGFlMzkKMHg2NDc0OQoweDRiMzNlCjB4NDZhNWIKMHgyYjVhYwoweDYxMjhhCjB4NjQ3NDkKMHg0ZjNhOQoweDYwZDMKMHgzNTgyYwoweGMxNGEKMHgxNWM0NAoweDY2MjNkCjB4MmI1YWMKMHgyNWRjYwoweDJiNWFjCjB4ZGMzCjB4ZGMzCjB4NWFmNTcKMHhhZTM5CjB4NjQ3NDkKMHg1YjZlMAoweDQwNjQ1CjB4NjQ3NDkKMHhjMTRhCjB4MmI1YWMKMHg0ZGI3ZAoweDlhMzk=",
    "signature": b"MHgxOTQxMAoweDE5NDEwCjB4MTRiYWIKMHg2MTZjCjB4ZGFhCjB4MTJkMmIKMHgxOTQxMAoweDEyZDJiCjB4NjE2YwoweDEyZDJiCjB4MTk0MTAKMHg2OGEzCjB4NjhhMwoweDYxNmMKMHg2OGEzCjB4NjhhMwoweDE1MTA5CjB4MTk0MTAKMHgxNGEyNAoweGU4MWYKMHgxNDIyOAoweDE0YTI0CjB4MTk2NmQKMHgxOTY2ZAoweGIyMGEKMHgxOTY2ZAoweDE2YzY5CjB4MTRiYWIKMHgxNDIyOAoweDExMjYyCjB4ZTgxZgoweDE5NDEwCjB4ZGFhCjB4MTJkMmIKMHgxNGJhYgoweDE5MTI5CjB4MTEyNjIKMHgxOTEyOQoweDE5NDEwCjB4MTJkMmIKMHgxNTEwOQoweGU4MWYKMHgxMTI2MgoweDEyZDJiCjB4MTk2NmQKMHgxNmM2OQoweGRhYQoweGIyMGEKMHgxNDIyOAoweDEyZDJiCjB4MTEyNjIKMHgxOTEyOQoweDE1MTA5CjB4ZTgxZgoweDE2YzY5CjB4MTk0MTAKMHgxOTEyOQoweDYxNmMKMHgxNGEyNAoweDE0YTI0CjB4MTk1YTkKMHgxNTEwOQoweDE5NDEwCjB4MTJkMmI=",
}

### Richael's Verification

Now, what Richael will do is:
- Use her own **private** key to decrypt **msg**
- Use Stephen's **public** key to decrypt **signature**
- SHA256 Hash the decrypted **msg** to reproduce the **signature**
- Compare the 2 **signatures** and see if they match:
    - if they do, the message is from Stephen and it is intact.
    - if not, something dodgy happened.

In [77]:
# RSA decode msg
decry_msg = [
    int(i, 16) ** private_key[1] % private_key[0]
    for i in base64.b64decode(message_to_Richael["msg"]).decode("utf8").split("\n")
]
decry_msg = "".join([chr(i) for i in decry_msg])
print(decry_msg)

Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)


In [78]:
# RSA decode sig
decry_sig = [
    int(i, 16) ** public_key_stephen[1] % public_key_stephen[0]
    for i in base64.b64decode(message_to_Richael["signature"]).decode("utf8").split("\n")
]
print(decry_sig)
print("".join([chr(i) for i in decry_sig]))

[50, 50, 101, 100, 99, 57, 50, 57, 100, 57, 50, 49, 49, 100, 49, 49, 48, 50, 102, 55, 97, 102, 56, 56, 98, 56, 53, 101, 97, 52, 55, 50, 99, 57, 101, 54, 52, 54, 50, 57, 48, 55, 52, 57, 56, 53, 99, 98, 97, 57, 52, 54, 48, 55, 53, 50, 54, 100, 102, 102, 51, 48, 50, 57]
22edc929d9211d1102f7af88b85ea472c9e64629074985cba94607526dff3029


In [82]:
# Compare Digital Signature
sha256(
    b"Hello Richael. Let's meet at 6:25 pm tomorrow at the airport Gate5, Terminal 2 --Stephen :)"
).hexdigest() == "".join([chr(i) for i in decry_sig])

True

In [92]:
assert sha256(b"Hello Richael. Let's meet at 6:25").hexdigest() == "".join(
    [chr(i) for i in decry_sig]
), "[Attention]: Dodgy thing happened with this message!"

AssertionError: [Attention]: Dodgy thing happened with this message!

For Joseph, he can also have Stephen's **public key**, intercept and decrypt the signature. However, if now he wants to alter the message ("random" stuff) without being found out, he must alter the Hash ("22edc929d9211d1102f7af88b85ea472c9e64629074985cba94607526dff3029") as well and **make sure they match**. Without decrypting the original message, it is infeasible to generated its Hash. Therefore Joseph has 2 options: alter the message and being found out, or leave message as is.

## Digital Signature

There are 2 problems with the encryption and decryption process described above:  

1. What if Joseph pretends to be Stephen, and sends her another message encrypted with Richaels **public key**, saying "Hello Richael. Let's meet at 6:25 am the day after tomorrow at the train station Gate 1 :)". How does Richael know the message is truly from Stephen, or being altered by Joseph.

*Answer*: this is solved by Digital Signature

2. Joseph could totally generate his own key pair and pretend to be Richael. When Richael's friends receive "her" **public key**, how do they know it truly comes from Richael, rather than Joseph?

*Answer*: this is solved by Certificate

## Reference

- https://blog.vrypan.net/2013/08/28/public-key-cryptography-for-non-geeks/
- https://www.ruanyifeng.com/blog/2013/06/rsa_algorithm_part_one.html
- https://us-cert.cisa.gov/ncas/tips/ST04-018#:~:text=Digital%20signatures%20work%20by%20proving,using%20the%20sender's%20private%20key.